# Reading the dataset

In [ ]:
import pandas as pd 

df_Eng = pd.read_csv('/content/New_English_Cleaned.csv')
df_Arb = pd.read_excel('/content/ArabicDataCleaned.xlsx')
df_Heb  = pd.read_excel('/content/HebrewDataCleaned.xlsx')


Spliting the datasets into 80% train and 20% test

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# train_Arb,test_Arb= train_test_split(df_Arb, test_size=0.2, random_state=1)

# train_Heb,test_Heb= train_test_split(df_Heb, test_size=0.2, random_state=1)

# train_Eng,test_Eng= train_test_split(df_Eng, test_size=0.2, random_state=1)


In [ ]:
train_Eng=pd.read_excel('train_Eng_SA.xlsx')
test_Eng=pd.read_excel('test_Eng_SA.xlsx')

train_Arb=pd.read_excel('train_Arb_SA.xlsx')
test_Arb=pd.read_excel('test_Arb_SA.xlsx')

train_Heb=pd.read_excel('train_Heb_SA.xlsx')
test_Heb=pd.read_excel('test_Heb_SA.xlsx')

#Mbert

##### Importing mBert tokenizer

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 32.7 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 6.6 MB 63.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer

max_length_test = 40
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)


Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

##### Model initialization

In [ ]:
from transformers import TFBertForSequenceClassification


model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##### Maping tokens into BOW embeddings

In [ ]:
import tensorflow as tf


# map to the expected input to TFBertForSequenceClassification 
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label



def encode_examples(ds):

  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []


  for index, row in ds.iterrows():
    bert_input=tokenizer.encode_plus(
                        str(row['text']),                      
                        add_special_tokens = True,
                        max_length = max_length_test, 
                        pad_to_max_length = True, 
                        return_attention_mask = True,
                        
              )


    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
 
    

    if row['label']==0 :
      label_list.append([0])
    else:
      label_list.append([1])

  #return input_ids_list
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)


##### Encoding the dataset

In [ ]:
batch_size=16

# train dataset
train = pd.concat([df_Heb,df_Eng,train_Arb])
train_encoded = encode_examples(train).shuffle(10000).batch(batch_size)

# test dataset
test  = test_Arb 
test_encoded  = encode_examples(test).shuffle(10000).batch(batch_size)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


##### Training the model

In [ ]:
learning_rate = 2e-5
number_of_epochs = 3

model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

bert_history = model.fit(train_encoded, epochs=number_of_epochs)

Epoch 1/3
1614/1614 [==============================] - 321s 190ms/step - loss: 0.6526 - accuracy: 0.6565
Epoch 2/3
1614/1614 [==============================] - 306s 190ms/step - loss: 0.6399 - accuracy: 0.6688
Epoch 3/3
1614/1614 [==============================] - 305s 189ms/step - loss: 0.6725 - accuracy: 0.6395


##### Evaluate the model


In [ ]:
loss, accuracy = model.evaluate(test_encoded)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

66/66 [==============================] - 6s 57ms/step - loss: 0.6883 - accuracy: 0.5881
Loss: 0.6882796883583069
Accuracy: 0.5881226062774658


***With SA double filtering***

In [ ]:
x = model.predict(test_encoded)

In [ ]:
preds_flat = np.argmax(x.logits, axis=1).flatten()

In [ ]:
preds_flat

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
def Max_SA(Pos ,Neg ,Net):
  if Pos > Neg and Pos > Net:
    return 'positive'
  if Neg > Pos and Neg > Net:
    return 'negative'
  if Net > Neg and Net > Pos:
    return 'neutral'

def updateSA(df, labels):

  c = 0
  for index, row in df.iterrows():
    Neg_SA = row['negative'] #25785
    Pos_SA = row['positive'] #19737
    Nat_SA = row['neutral'] #52546

    if(Max_SA(Pos_SA,Neg_SA,Nat_SA)== 'positive'):
      labels[c] = 0
    c += 1


In [ ]:
updateSA(test_Arb, preds_flat)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
accuracy_score(test_Arb['label'].to_list(),preds_flat)

0.7233695652173913

#Machine Learning

In [ ]:
def print_scores(test_labels,model_preds):
  accuracy=accuracy_score(test_labels,model_preds)
  precision = precision_score(test_labels,model_preds, average='binary')
  recall = recall_score(test_labels,model_preds, average='binary')
  f1_score = f1_score(test_labels,model_preds, average='binary')
  print('Accuracy: %.3f' % accuracy ,'Precision: %.3f' % precision,'Recall: %.3f' % recall,'F-Measure: %.3f' % f1_score)


#####Machine Learning + N-grams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
import numpy as np


train=pd.concat([train_Heb,df_Eng,df_Arb])
test=test_Heb


train_text = [str(i) for i in train['text'] ]
test_text  = [str(i) for i in test['text']  ]
all_text = train_text+test_text

#ngram_vectorizer = TfidfVectorizer()
ngram_vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,3))

ngram_vectorizer.fit(all_text)
train_features = ngram_vectorizer.transform(train_text)
test_features = ngram_vectorizer.transform(test_text)

train_target=train['label']
test_target=test['label']

classifiers=[RandomForestClassifier(random_state=1)
              ,LogisticRegression(C=0.1, solver='sag',random_state=1)
              ,svm.SVC()
            ]

for classifier in classifiers : 
  classifier.fit(train_features, train_target)
  model_pred=classifier.predict(test_features)
  print_scores(test_target,model_pred)


#####Machine Learning + BOW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
import numpy as np


train=pd.concat([train_Heb,df_Eng,df_Arb])
test=test_Heb


train_text = [str(i) for i in train['text'] ]
test_text  = [str(i) for i in test['text']  ]
all_text = train_text+test_text

bow_vectorizer = TfidfVectorizer()


bow_vectorizer.fit(all_text)
train_features = bow_vectorizer.transform(train_text)
test_features = bow_vectorizer.transform(test_text)

train_target=train['label']
test_target=test['label']

classifiers=[RandomForestClassifier(random_state=1)
              ,LogisticRegression(C=0.1, solver='sag',random_state=1)
              ,svm.SVC()
            ]

for classifier in classifiers : 
  classifier.fit(train_features, train_target)
  model_pred=classifier.predict(test_features)
  print_scores(test_target,model_pred)


#####Machine Learning + Sentence Vectors Mbert

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
bert_transformers = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [ ]:
import itertools
import re

def cleanText(text):
    text=str(text)
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'re", " are", text)
    text = text.strip().lower()
    filters='"#$%&*+<=>@[\\]^_`{|}~\t\n'
    text = re.sub(r'\!+', '!', text)
    text = re.sub(r'\?+', '?', text)
    text = re.sub(r'\( *\)', ' ', text)
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text


def mean(z): # used for BERT (word version) and Word2Vec
    return sum(itertools.chain(z))/len(z)

def embeddToBERT(text):
    sentences = re.split('!|\?|\.',text)
    sentences = list(filter(None, sentences)) 
    result = bert_transformers.encode(sentences)
    feature = [mean(x) for x in zip(*result)]
  
    return feature

In [ ]:
test_Arb=test_Arb.drop(1613)
train_Arb=train_Arb.drop([4002,4891,1613,6920,1512])

In [ ]:
test_Heb['clean_text_bert'] = test_Heb['text'].apply(cleanText)
train_Heb['clean_text_bert'] = train_Heb['text'].apply(cleanText)

test_Arb['clean_text_bert'] = test_Arb['text'].apply(cleanText)
train_Arb['clean_text_bert'] = train_Arb['text'].apply(cleanText)

test_Eng['clean_text_bert'] = test_Eng['text'].apply(cleanText)
train_Eng['clean_text_bert'] = train_Eng['text'].apply(cleanText)

In [ ]:
He_train_feature = train_Heb['clean_text_bert'].apply(embeddToBERT)
He_test_feature = test_Heb['clean_text_bert'].apply(embeddToBERT)

In [ ]:
En_train_feature = train_Eng['clean_text_bert'].apply(embeddToBERT)
En_test_feature = test_Eng['clean_text_bert'].apply(embeddToBERT)

In [ ]:
Ar_train_feature = train_Arb['clean_text_bert'].apply(embeddToBERT)
Ar_test_feature = test_Arb['clean_text_bert'].apply(embeddToBERT)

With SA labels (optional)

In [ ]:
#add SA feature to train 
train_He_SA=[]
negative=train_Arb['negative'].to_list()
positive=train_Arb['positive'].to_list()
neutral=train_Arb['neutral'].to_list()
for i in range(train_Arb.shape[0]):  
  train_He_SA.append(Ar_train_feature.iloc[i]+[negative[i],positive[i],neutral[i]])
  
Ar_train_feature = train_He_SA
Ar_train_feature = pd.Series(Ar_train_feature)


#add SA feature to test 
test_He_SA=[]
negative=test_Arb['negative'].to_list()
positive=test_Arb['positive'].to_list()
neutral=test_Arb['neutral'].to_list()
for i in range(test_Arb.shape[0]):
  test_He_SA.append(Ar_test_feature.iloc[i]+[negative[i],positive[i],neutral[i]])

Ar_test_feature = test_He_SA
Ar_test_feature = pd.Series(Ar_test_feature)

In [ ]:
feature = [x for x in He_train_feature.transpose()]
He_train = np.asarray(feature)

feature = [x for x in He_test_feature.transpose()]
He_test = np.asarray(feature)

print(He_test.shape)

(1044, 768)


In [ ]:
feature = [x for x in Ar_train_feature.transpose()]
Ar_train = np.asarray(feature)

feature = [x for x in Ar_test_feature.transpose()]
Ar_test = np.asarray(feature)

print(Ar_train.shape)

(7351, 768)


In [ ]:
feature = [x for x in En_train_feature.transpose()]
En_train = np.asarray(feature)

feature = [x for x in En_test_feature.transpose()]
En_test = np.asarray(feature)

print(En_train.shape)

(10592, 768)


In [ ]:
Ar_train_labels=train_Arb['label'].to_list()
He_train_labels=train_Heb['label'].to_list()
En_train_labels=train_Eng['label'].to_list()

Ar_test_labels=test_Arb['label'].to_list()
He_test_labels=test_Heb['label'].to_list()
En_test_labels=test_Eng['label'].to_list()

In [ ]:
train_features=list(Ar_train) + list(Ar_test) + list(En_train) + list(En_test) + list(He_train)
train_targets=Ar_train_labels + Ar_test_labels + En_train_labels + En_test_labels + He_train_labels

test_features=list(He_test)
test_targets=He_test_labels

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
import numpy as np


classifiers=[RandomForestClassifier(random_state=1)
              ,LogisticRegression(C=0.1, solver='sag',random_state=1)
              ,svm.SVC()
            ]


for model in classifiers : 
  model.fit(train_features,train_targets)
  y_pred = model.predict(test_features)
  print_scores(test_targets,y_pred)  


#Resampling

In [ ]:
df=df_Eng

In [ ]:
count_class_0, count_class_1 = df.label.value_counts()

# Divide by class
df_class_0 = df[df['label'] == 0]
df_class_1 = df[df['label'] == 1]
print("Before:\n",df.label.value_counts())

Before:
 0    9748
1    1196
Name: label, dtype: int64


###Under-sampling

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
df_under = pd.concat([df_class_0_under, df_class_1], axis=0)


print("After:\n",df_under.label.value_counts())

After:
 1    1196
0    1196
Name: label, dtype: int64


###Over-sampling

In [ ]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_over = pd.concat([df_class_0, df_class_1_over], axis=0)


print("After:\n",df_over.label.value_counts())

# English dataset preprocessing

In [ ]:
import re
import nltk
nltk.download('stopwords')

# Invert capital letters to small letters
df_Eng['text']=df_Eng['text'].str.lower()

# Removing punctuation marks and special characters
def Clean(word):
    return re.sub('[^A-Za-z]+', '', word)
df_Eng['text'] = df_Eng['text'].apply(lambda y: " ".join(Clean(x) for x in y.split()))

# Removing stopwords
stop = stopwords.words('english')
df_Eng['text'] = df_Eng['text'].apply(lambda y: " ".join(x for x in y.split() if x not in stop))

df_Eng['text'] = df_Eng['text'].apply(lambda y: " ".join(x for x in y.split() if len(x)>2))

# Arabic dataset preprocessing

In [ ]:
df_Arb.head()

,Tweet,Offensive,Hate,class
0,الحمدلله يارب فوز مهم يا زمالك.. كل الدعم ليكم...,NOT_OFF,NOT_HS,0.0
1,فدوه يا بخت فدوه يا زمن واحد منكم يجيبه,NOT_OFF,NOT_HS,0.0
2,RT @USER: يا رب يا واحد يا أحد بحق يوم الاحد ا...,OFF,HS,1.0
3,RT @USER: #هوا_الحرية يا وجع قلبي عليكي يا امي...,NOT_OFF,NOT_HS,0.0
4,يا بكون بحياتك الأهم يا إما ما بدي أكون 🎼,NOT_OFF,NOT_HS,0.0


In [ ]:
import os
import joblib
import re
import string

def clean(df):
    df = remove_diacritics(df)
    df = normalize_arabic(df)
    df = remove_punctuations(df)
    df = remove_repeating_char(df)
    df = remove_english_word_and_numbers(df)
    df = clean_space(df)
    df['Tweet']=df['Tweet'].apply(lambda x :remove_emoji(str(x)))
    return df


arabic_punctuations = '''`÷« »×؛<>٩٨'٧٦٥٤٣٢١٠_()↗*•&^%][ـ،/:"؟.,'{}⋮≈~¦+|٪!”…“–ـ/[]%=#*+\\•~@£·_{}©^®`→°€™›♥←×§″′Â█à…“★”–●â►−¢¬░¶↑±▾	═¦║―¥▓—‹─▒：⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡₹´'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def remove_diacritics(df):
    df['Tweet'] = df['Tweet'].apply(lambda x: _remove_diacritics(x))
    return df


def _remove_diacritics(x):
    x = str(x)
    x = re.sub(arabic_diacritics, '', x)
    return x


def normalize_arabic(df):
    df['Tweet'] = df['Tweet'].apply(lambda x: _normalize_arabic(x))
    return df


def _normalize_arabic(x):
    x = str(x)
    # added space around puncts after replace
    x = re.sub("[إأآا]", "ا", x)
    x = re.sub("ى", "ي", x)
    x = re.sub("ؤ", "ء", x)
    x = re.sub("ئ", "ء", x)
    x = re.sub("ة", "ه", x)
    x = re.sub("گ", "ك", x)
    return x


def remove_punctuations(df):
    df['Tweet'] = df['Tweet'].apply(lambda x: _remove_punctuations(x))
    return df


def _remove_punctuations(x):
    x = str(x)
    translator = str.maketrans(punctuations_list, ' '*len(punctuations_list))
    return x.translate(translator)


def remove_repeating_char(df):
    df['Tweet'] = df['Tweet'].apply(lambda x: _remove_repeating_char(x))
    return df


def _remove_repeating_char(x):
    x = str(x)
    return re.sub(r'(.)\1+', r'\1', x)


def remove_english_word_and_numbers(df):
    df['Tweet'] = df['Tweet'].apply(
        lambda x: _remove_english_word_and_numbers(x))
    return df


def _remove_english_word_and_numbers(x):
    x = str(x)
    return re.sub(r'[a-zA-Z0-9]+', '', x)


def clean_space(df):
    compiled_re = re.compile(r"\s+")
    df['Tweet'] = df["Tweet"].apply(lambda x: _clean_space(x, compiled_re))
    return df


def _clean_space(x, compiled_re):
    return compiled_re.sub(" ", x)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
clean(df_Arb)

,Tweet,Offensive,Hate,class
0,الحمدله يارب فوز مهم يا زمالك كل الدعم ليكم يا...,NOT_OFF,NOT_HS,0.0
1,فدوه يا بخت فدوه يا زمن واحد منكم يجيبه,NOT_OFF,NOT_HS,0.0
2,يا رب يا واحد يا احد بحق يوم الاحد ان تهلك بن...,OFF,HS,1.0
3,هوا الحريه يا وجع قلبي عليكي يا امي اله لا يح...,NOT_OFF,NOT_HS,0.0
4,يا بكون بحياتك الاهم يا اما ما بدي اكون,NOT_OFF,NOT_HS,0.0
...,...,...,...,...
9191,تلحس اجري شقفه جحش,OFF,HS,1.0
9192,كول هوا يا هوا عميل كلب,OFF,HS,1.0
9193,طيب حضرتك شو عمبتقدم لبلدك غير لغه الصرامي,OFF,HS,1.0
9194,وانا لح قلك مارون غنيم ماتواخذني تلحس رجلي مع ...,OFF,HS,1.0


In [ ]:
df_Arb=df_Arb.rename(columns={"Tweet": "text"})

In [ ]:
df_Arb['label']=[0 if row['Hate']=='NOT_HS' else 1 for index,row in df_Arb.iterrows()]
df_Arb

,text,Offensive,Hate,class,label
0,الحمدله يارب فوز مهم يا زمالك كل الدعم ليكم يا...,NOT_OFF,NOT_HS,0.0,0
1,فدوه يا بخت فدوه يا زمن واحد منكم يجيبه,NOT_OFF,NOT_HS,0.0,0
2,يا رب يا واحد يا احد بحق يوم الاحد ان تهلك بن...,OFF,HS,1.0,1
3,هوا الحريه يا وجع قلبي عليكي يا امي اله لا يح...,NOT_OFF,NOT_HS,0.0,0
4,يا بكون بحياتك الاهم يا اما ما بدي اكون,NOT_OFF,NOT_HS,0.0,0
...,...,...,...,...,...
9191,تلحس اجري شقفه جحش,OFF,HS,1.0,1
9192,كول هوا يا هوا عميل كلب,OFF,HS,1.0,1
9193,طيب حضرتك شو عمبتقدم لبلدك غير لغه الصرامي,OFF,HS,1.0,1
9194,وانا لح قلك مارون غنيم ماتواخذني تلحس رجلي مع ...,OFF,HS,1.0,1


In [ ]:
df_Arb.to_excel("ArabicDataCleaned.xlsx")

#Hebrew dataset preprocessing

In [ ]:
import numpy as np
import spacy
import re
import joblib

In [ ]:
df_HW

,text,label
0,"התנהגו בצורה מכוערת מאוד, ניבולי פה, מה זה?.? ...",0
1,יהיה זכרו ברוך!,0
2,ישועת השם כהרף עין אמן,0
3,תמיד כל כך עניינית וישרה. כן ירבו כמוך ...,0
4,@LittleW_SAndS בן זונה אפס,1
...,...,...
5212,למה למישהוא היה ספק אחרת? מחבל נישאר מחבל,1
5213,עיתונאי רוצח תיזהר,0
5214,איש מביש ודוחה,1
5215,"""את חיה בסרט אם את חושבת שבוחרי הליכוד יעברו ל...",0


In [ ]:
df_HW["text"]=df_HW["text"].apply(lambda x :re.sub(r'[^א-ת]+',' ',x))
df_HW

,text,label
0,התנהגו בצורה מכוערת מאוד ניבולי פה מה זה בושה ...,0
1,יהיה זכרו ברוך,0
2,ישועת השם כהרף עין אמן,0
3,תמיד כל כך עניינית וישרה כן ירבו כמוך בישראל,0
4,בן זונה אפס,1
...,...,...
5212,למה למישהוא היה ספק אחרת מחבל נישאר מחבל,1
5213,עיתונאי רוצח תיזהר,0
5214,איש מביש ודוחה,1
5215,את חיה בסרט אם את חושבת שבוחרי הליכוד יעברו ל...,0


In [ ]:
stop_word=[
'אני',
'את',
'אתה',
'אנחנו',
'אתן',
'אתם',
'הם',
'הן',
'היא',
'הוא',
'שלי',
'שלו',
'שלך',
'שלה',
'שלנו',
'שלכם',
'שלכן',
'שלהם',
'שלהן',
'לי',
'לו',
'לה',
'לנו',
'לכם',
'לכן',
'להם',
'להן',
'אותה',
'אותו',
'זה',
'זאת',
'אלה',
'אלו',
'תחת',
'מתחת',
'מעל',
'בין',
'עם',
'עד',
'נגר',
'על',
'אל',
'מול',
'של',
'אצל',
'כמו',
'אחר',
'אותו',
'בלי',
'לפני',
'אחרי',
'מאחורי',
'עלי',
'עליו',
'עליה',
'עליך',
'עלינו',
'עליכם',
'לעיכן',
'עליהם',
'עליהן',
'כל',
'כולם',
'כולן',
'כך',
'ככה',
'כזה',
'זה',
'זות',
'אותי',
'אותה',
'אותם',
'אותך',
'אותו',
'אותן',
'אותנו',
'ואת',
'את',
'אתכם',
'אתכן',
'איתי',
'איתו',
'איתך',
'איתה',
'איתם',
'איתן',
'איתנו',
'איתכם',
'איתכן',
'יהיה',
'תהיה',
'היתי',
'היתה',
'היה',
'להיות',
'עצמי',
'עצמו',
'עצמה',
'עצמם',
'עצמן',
'עצמנו',
'עצמהם',
'עצמהן',
'מי',
'מה',
'איפה',
'היכן',
'במקום שבו',
'אם',
'לאן',
'למקום שבו',
'מקום בו',
'איזה',
'מהיכן',
'איך',
'כיצד',
'באיזו מידה',
'מתי',
'בשעה ש',
'כאשר',
'כש',
'למרות',
'לפני',
'אחרי',
'מאיזו סיבה',
'הסיבה שבגללה',
'למה',
'מדוע',
'לאיזו תכלית',
'כי',
'יש',
'אין',
'אך',
'מנין',
'מאין',
'מאיפה',
'יכל',
'יכלה',
'יכלו',
'יכול',
'יכולה',
'יכולים',
'יכולות',
'יוכלו',
'יוכל',
'מסוגל',
'לא',
'רק',
'אולי',
'אין',
'לאו',
'אי',
'כלל',
'נגד',
'אם',
'עם',
'אל',
'אלה',
'אלו',
'אף',
'על',
'מעל',
'מתחת',
'מצד',
'בשביל',
'לבין',
'באמצע',
'בתוך',
'דרך',
'מבעד',
'באמצעות',
'למעלה',
'למטה',
'מחוץ',
'מן',
'לעבר',
'מכאן',
'כאן',
'הנה',
'הרי',
'פה',
'שם',
'אך',
'ברם',
'שוב',
'אבל',
'מבלי',
'בלי',
'מלבד',
'רק',
'בגלל',
'מכיוון',
'עד',
'אשר',
'ואילו',
'למרות',
'אס',
'כמו',
'כפי',
'אז',
'אחרי',
'כן',
'לכן',
'לפיכך',
'מאד',
'עז',
'מעט',
'מעטים',
'במידה',
'שוב',
'יותר',
'מדי',
'גם',
'כן',
'נו',
'אחר',
'אחרת',
'אחרים',
'אחרות',
'אשר',
'או'
]

In [ ]:
df_HW["text"] = df_HW["text"].apply(lambda x: ' '.join([t for t in x.split() if t not in stop_word]))
df_HW

,text,label
0,התנהגו בצורה מכוערת מאוד ניבולי בושה לומר ללימ...,0
1,זכרו ברוך,0
2,ישועת השם כהרף עין אמן,0
3,תמיד עניינית וישרה ירבו כמוך בישראל,0
4,בן זונה אפס,1
...,...,...
5212,למישהוא ספק מחבל נישאר מחבל,1
5213,עיתונאי רוצח תיזהר,0
5214,איש מביש ודוחה,1
5215,חיה בסרט חושבת שבוחרי הליכוד יעברו למרצ שיקרה ...,0


In [ ]:
def remove_repeating_char(data):
    data['text'] = data['text'].apply(lambda x: _remove_repeating_char(x))
    
def _remove_repeating_char(x):
    x = str(x)
    return re.sub(r'(.)\1+', r'\1', x)
  
remove_repeating_char(df_HW)
df_HW

,text,label
0,התנהגו בצורה מכוערת מאוד ניבולי בושה לומר לימו...,0
1,זכרו ברוך,0
2,ישועת השם כהרף עין אמן,0
3,תמיד ענינית וישרה ירבו כמוך בישראל,0
4,בן זונה אפס,1
...,...,...
5212,למישהוא ספק מחבל נישאר מחבל,1
5213,עיתונאי רוצח תיזהר,0
5214,איש מביש ודוחה,1
5215,חיה בסרט חושבת שבוחרי הליכוד יעברו למרצ שיקרה ...,0


In [ ]:
df_HW["text"]=df_HW["text"].apply(lambda x:' '.join(x.split()))
df_HW

,text,label
0,התנהגו בצורה מכוערת מאוד ניבולי בושה לומר לימו...,0
1,זכרו ברוך,0
2,ישועת השם כהרף עין אמן,0
3,תמיד ענינית וישרה ירבו כמוך בישראל,0
4,בן זונה אפס,1
...,...,...
5212,למישהוא ספק מחבל נישאר מחבל,1
5213,עיתונאי רוצח תיזהר,0
5214,איש מביש ודוחה,1
5215,חיה בסרט חושבת שבוחרי הליכוד יעברו למרצ שיקרה ...,0


In [ ]:
df_HW.to_excel("NewDataHebrewCleaned.xlsx")